In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
import sys
import copy
import os
import string
from sklearn.preprocessing import MinMaxScaler
import operator

In [ ]:
path = r'C:\Users\supri\Downloads\HW7\books'
final_file = open('full_text.txt', 'w')
for text_file in os.listdir(path):
    with open(path + '\\' + text_file, 'r', 
              errors='ignore') as file_data:
        for line in file_data:
            final_file.write(line)

final_file.close()
text = open('MLOE.txt', 'r').read()
print('length of the file is:',len(text))

In [ ]:
# trimedText = copy.copy(orginalText)
# charSet = set(orginalText)
# if ignore_case:
lower_text = text.lower()
char_set = set(text.lower())
# if remove_punctuation:
char_set = char_set.difference(set(string.punctuation))
lower_text = lower_text.translate(str.maketrans('', '', string.punctuation))
# return trimedText, charSet

In [ ]:
char_dict = {}
for index, char in enumerate(sorted(char_set)):
    char_dict[char] = index
scaled_dict = {}
scaled_values = MinMaxScaler().fit_transform(np.array(list(char_dict.values())).reshape(-1, 1))
for index in range(len(scaled_values)):
    scaled_dict[list(char_dict.keys())[index]] = scaled_values[index][0]

In [ ]:
window_size = 99
X = []
y = []
for i in range(0, len(lower_text) - window_size, 1):
    temp_line = lower_text[i:i + window_size]
    last_char = lower_text[i + window_size]
    X.append([scaled_dict[char] for char in temp_line])
    y.append(char_dict[last_char])

In [ ]:
X = np.reshape(X, (len(X), window_size, 1))
y = keras.utils.to_categorical(y)
print(X.shape)
print(y.shape)

In [ ]:
memory_units = 256

keras_model = keras.models.Sequential([
    keras.layers.LSTM(units=memory_units,
                      input_shape=(X.shape[1], X.shape[2])),
    keras.layers.Dense(y.shape[1], activation="softmax"),
])
keras_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))
keras_model.summary()

In [ ]:
epochs = 1
output_folder = "./keras_files"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
keras_prefix = os.path.join(output_folder, 'keras_{epoch:02d}.hdf5')
keras_callback = keras.callbacks.ModelCheckpoint(filepath = keras_prefix,monitor='loss',save_weights_only= True,mode='min')
keras_early_stopping = keras.callbacks.EarlyStopping(patience = 5, min_delta = 1e-4)
keras_hist = keras_model.fit(X, y, epochs=epochs, batch_size=128, 
                    callbacks=[keras_callback, keras_early_stopping])

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
pip install tensorflow-gpu

  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      Successfully uninstalled tensorboard-2.0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow 2.0.0 has requirement tensorboard<2.1.0,>=2.0.0, but you'll have tensorboard 2.2.2 which is incompatible.


In [ ]:
pip uninstall tensorflow==2.0